# Download Data from WRDS

In [1]:
!pip install wrds

In [2]:
!pip install pyarrow

In [3]:
import wrds
import pandas as pd
from datetime import datetime

import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
sp500_df = pd.read_csv("/home/sagemaker-user/capstone-2024-summer/data/security_master.csv")

sp500_df.head()

,cusip,PERMNO,PERMCO,HSHRCD,DLSTCD,HTICK,HCOMNAM,HTSYMBOL,HNAICS,HPRIMEXC,...,NUMDEL,NUMNDI,BEGDAT,ENDDAT,BEGPRC,ENDPRC,BEGRET,ENDRET,BEGVOL,ENDVOL
0,00130H10,76712,10996,11,100,AES,A E S CORP,AES,221118,N,...,1,68,06/26/1991,12/29/2023,06/26/1991,12/29/2023,06/26/1991,12/29/2023,06/26/1991,12/29/2023
1,00206R10,66093,21645,11,100,T,A T & T INC,T,517312,N,...,1,0,02/16/1984,12/29/2023,02/16/1984,12/29/2023,02/16/1984,12/29/2023,02/16/1984,12/29/2023
2,00507V10,79678,12499,11,233,NaN,ACTIVISION BLIZZARD INC,ATVI,513210,Q,...,1,1968,10/22/1993,10/12/2023,10/22/1993,10/12/2023,10/22/1993,10/12/2023,10/22/1993,10/12/2023
3,00724F10,75510,8476,11,100,ADBE,ADOBE INC,ADBE,511210,Q,...,1,2185,08/13/1986,12/29/2023,08/13/1986,12/29/2023,08/13/1986,12/29/2023,08/13/1986,12/29/2023
4,00971T10,87299,17300,11,100,AKAM,AKAMAI TECHNOLOGIES INC,AKAM,511210,Q,...,1,1722,10/29/1999,12/29/2023,10/29/1999,12/29/2023,10/29/1999,12/29/2023,10/29/1999,12/29/2023


In [ ]:
# Connect to WRDS
db = wrds.Connection()

# Set date range
start_date = '2018-01-01'
end_date = '2023-12-31'

sp500_permnos = tuple(sp500_df['PERMNO'].unique())

# Main query
main_query = f"""
SELECT a.*,
       c.*
FROM crsp.dsf a
LEFT JOIN crsp.ccmxpf_linktable b
ON a.permno = b.lpermno
AND b.linktype IN ('LC', 'LU')
AND b.linkprim IN ('P', 'C')
AND a.date BETWEEN b.linkdt AND COALESCE(b.linkenddt, '{end_date}')
LEFT JOIN LATERAL (
    SELECT *
    FROM comp.fundq c2
    WHERE c2.gvkey = b.gvkey
      AND c2.datadate <= a.date
      AND c2.datadate BETWEEN '{start_date}' AND '{end_date}'
    ORDER BY c2.datadate DESC
    LIMIT 1
) c ON TRUE
WHERE a.date BETWEEN '{start_date}' AND '{end_date}'
AND a.permno IN {sp500_permnos}
"""

print("Executing query...")
data = db.raw_sql(main_query)

print("Query executed. Saving to CSV...")

# Save to CSV
data.to_csv('crsp_compustat_merged_2018_2023.csv', index=False)

# Close the connection
db.close()

print("Data extraction and merging complete. File saved as CSV.")

# Print column names for reference
print("\nColumns in the merged dataset:")
print(data.columns.tolist())

Enter your WRDS username [sagemaker-user]: jennasparks
Enter your password: ········


WRDS recommends setting up a .pgpass file.


Create .pgpass file now [y/n]?:  n


You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done
Executing query...


## Check the downloaded data

In [ ]:
file_path = "/home/sagemaker-user/capstone-2024-summer/src/jenna/crsp_compustat_merged_2018_2023.csv"
# Read the CSV file with specified data types
df = pd.read_csv(file_path, low_memory=False)

In [ ]:
df.head(10)

In [ ]:
df.columns